In [1]:
# First things first...

import cv2
import random
import pandas as pd
import statistics

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Taken from pretrained_example.py
import os
import pickle
import PIL.Image
import numpy as np
import dnnlib
import dnnlib.tflib as tflib
import config
from encoder.generator_model import Generator

# Off-the-shelf recognizer
import face_recognition

# Plotting
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image, ImageDraw, ImageFont

In [2]:
# Plot latent vectors of shape 18x512
def generate_image(latent_vector, size=512):
    latent_vector = latent_vector.reshape((1, 18, 512))
    generator.set_dlatents(latent_vector)
    img_array = generator.generate_images()[0]
    img = PIL.Image.fromarray(img_array, 'RGB')
#     return img.resize((1024, 1024))
    return img.resize((size, size))

In [3]:
def setup():
    tflib.init_tf()
    # Load pre-trained network.
    url = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ' # karras2019stylegan-ffhq-1024x1024.pkl
    with dnnlib.util.open_url(url, cache_dir=config.cache_dir) as f:
        _G, _D, Gs = pickle.load(f)
    generator = Generator(Gs, batch_size=1, randomize_noise=False) # -- RUNNING >1 TIMES THROWS ERROR
    fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    return [_G, _D, Gs, generator, fmt]

In [4]:
# Only run once.
[_G, _D, Gs, generator, fmt] = setup()

In [5]:
# # # # #
# subjects = {}
# generated_images = os.listdir('./generated_images/')
# for fname in generated_images:
#     subject = fname.split('d')[0]
#     lr = np.load('./latent_representations/'+fname.split('.')[0] + '.npy')
#     if subject in subjects:
#         subjects[subject]['generated_images'].append(fname)
#         subjects[subject]['latent_representations'].append(lr)            
#     else:
#         subjects[subject] = {
#             'generated_images': [fname],
#             'latent_representations': [lr]
#         }
#     if len(subjects[subject]['latent_representations'])==4:
#         sum_lr = np.zeros((18,512))
#         for lr in subjects[subject]['latent_representations']:
#             sum_lr += lr            
#         subjects[subject]['latent_representations_mean'] = sum_lr/4
# # # # # 
# with open('subjects.pkl','wb') as f:
#     pickle.dump(subjects, f)

In [6]:
with open('subjects.pkl','rb') as f:
    subjects = pickle.load(f)

In [7]:
len(subjects)

192

In [ ]:
# Is the average of each subject's 4-tuple still recognizable as the same person?

In [22]:
sub = '04261'

In [67]:
centroid_img = generate_image(subjects[sub]['latent_representations_mean'])
versus_img = [generate_image(lr) for lr in subjects[sub]['latent_representations']]

In [68]:
centroid_enc = face_recognition.face_encodings(np.array(centroid_img))

In [69]:
versus_enc = np.array([face_recognition.face_encodings(np.array(image)) for image in versus_img])

In [8]:
def interpolate_ensembles(sub1, sub2, steps=60, pkl_fname='interpolate_ensembles.pkl'):
    
    ensemble_sub1 = subjects[sub1]
    ensemble_sub2 = subjects[sub2]
    
    centroid1 = ensemble_sub1['latent_representations_mean']
    centroid2 = ensemble_sub2['latent_representations_mean']
    
    centroid_img1 = generate_image(centroid1)
    centroid_img2 = generate_image(centroid2)
    
    vec1_slim = np.reshape(centroid1, [1, centroid1.shape[0] * centroid1.shape[1]])
    vec2_slim = np.reshape(centroid2, [1, centroid2.shape[0] * centroid2.shape[1]])
    
    enc1 = face_recognition.face_encodings(np.array(centroid_img1))[0]
    enc2 = face_recognition.face_encodings(np.array(centroid_img2))[0]

    known_encodings = [enc1, enc2]
    
    images = []
    fr_distances = []
    
    z = np.empty((steps, vec1_slim.shape[1]))
    for i, alpha in enumerate(np.linspace(start=1.0, stop=0.0, num=steps)):

        # Linearly interpolate.
        z[i] = (alpha) * vec1_slim + (1.0-alpha) * vec2_slim

        # Reshape interpolated vector, and get interpolated image.
        curr_vec = np.reshape(z[i], [18, 512]) # back to original shape
        curr_img = np.array(generate_image(curr_vec))

        curr_enc = face_recognition.face_encodings(curr_img)[0]
        fr_distance = face_recognition.face_distance(known_encodings, curr_enc)

        images.append(np.array(curr_img))
        fr_distances.append(fr_distance)
        
    return_dict = {
        'images': images,
        'fr_distances': fr_distances,
    }
    
    return return_dict

In [9]:
# results = interpolate_ensembles('04261', '02463')

In [10]:
# results['fr_distances']

[array([4.12966288e-05, 8.74624020e-01]),
 array([0.03213133, 0.86791577]),
 array([0.03560125, 0.86434541]),
 array([0.06162209, 0.85767704]),
 array([0.06893946, 0.85276984]),
 array([0.09454193, 0.84726957]),
 array([0.09908839, 0.84008087]),
 array([0.11624539, 0.83835228]),
 array([0.13331088, 0.83546904]),
 array([0.15929139, 0.83365832]),
 array([0.17543489, 0.82975837]),
 array([0.19058066, 0.81975683]),
 array([0.20513742, 0.81471387]),
 array([0.22226042, 0.80795868]),
 array([0.23453634, 0.80348428]),
 array([0.25285345, 0.79397346]),
 array([0.2686081 , 0.78519981]),
 array([0.28471693, 0.77225984]),
 array([0.29800897, 0.75977539]),
 array([0.31045764, 0.74703414]),
 array([0.3309452 , 0.73480665]),
 array([0.34498794, 0.71667825]),
 array([0.36144759, 0.69733721]),
 array([0.37278683, 0.68418438]),
 array([0.3875843 , 0.66140655]),
 array([0.40447225, 0.64502694]),
 array([0.42802858, 0.61796615]),
 array([0.44514279, 0.59683473]),
 array([0.47184745, 0.57071031]),
 array

In [ ]:
# idx_sub1 = 0
# idx_sub2 = 0
# for i, pair in enumerate(rd['fr_distances']):
#     if pair[0] < 0.4:
#         idx_sub1 = i
#     if pair[1] > 0.4:
#         idx_sub2 = i+1

In [ ]:
# idx_sub1, idx_sub2